In [2]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

In [3]:
import sys
import os
# dirty method to add ocean direct SDK to path
sys.path.insert(1, os.path.abspath(r'C:\Program Files\Ocean Optics\OceanDirect SDK\Python'))
print(sys.path)

['C:\\ExperimentalData\\2024\\2024-09\\2024-09-09_K_spec_test', 'C:\\Program Files\\Ocean Optics\\OceanDirect SDK\\Python', 'C:\\ProgramData\\Anaconda3\\python39.zip', 'C:\\ProgramData\\Anaconda3\\DLLs', 'C:\\ProgramData\\Anaconda3\\lib', 'C:\\ProgramData\\Anaconda3', '', 'C:\\ProgramData\\Anaconda3\\lib\\site-packages', 'C:\\ProgramData\\Anaconda3\\lib\\site-packages\\win32', 'C:\\ProgramData\\Anaconda3\\lib\\site-packages\\win32\\lib', 'C:\\ProgramData\\Anaconda3\\lib\\site-packages\\Pythonwin', 'C:\\ProgramData\\Anaconda3\\lib\\site-packages\\IPython\\extensions', 'C:\\Users\\GMX\\.ipython']


In [18]:

# not sure yet
def get_spec_formatted(device, sn):
    try:
        #device.set_electric_dark_correction_usage(False);
        #device.set_nonlinearity_correction_usage(False);

        #10ms
        device.set_integration_time(10000);

        print("Reading spectra for dev s/n = %s" % sn, flush=True)
        for i in range(10):
            spectra = device.get_formatted_spectrum()
            print("spectra[100,200,300,400]: %d, %d, %d, %d" % (spectra[100], spectra[200], spectra[300], spectra[400]), flush=True)
    except OceanDirectError as e:
        logger.error(e.get_error_details())

In [4]:
from oceandirect.OceanDirectAPI import OceanDirectAPI, OceanDirectError, FeatureID
from oceandirect.od_logger import od_logger

# Initialise devices (proper?)
logger = od_logger()
od = OceanDirectAPI()
device_count = od.find_usb_devices()
device_ids = od.get_device_ids()
print("Found %d devices" % device_count)
for i in range(device_count):
    print("Device %d: %s" % (i, device_ids[i]))

# # not useful
# (major, minor, point) = od.get_api_version_numbers()
# print("API Version  : %d.%d.%d " % (major, minor, point))
# print("Total Device : %d     \n" % device_count)

# if device_count == 0:
#     print("No device found.")
# else:
#     for id in device_ids:
#         device       = od.open_device(id)
#         serialNumber = device.get_serial_number()

#         print("First Device : %d       " % id)
#         print("Serial Number: %s     \n" % serialNumber)

#         get_spec_formatted(device, serialNumber)

#         print("Closing device!\n")
#         od.close_device(id)

Found 1 devices
Device 0: 2


In [7]:
from oceandirect.OceanDirectAPI import OceanDirectAPI, OceanDirectError, FeatureID
from oceandirect.od_logger import od_logger


## need to update with choice of logger

####
# Initialisation simplified
####
od = OceanDirectAPI()
device_count = od.find_usb_devices()

# Let's assume there is only one device connected
device_ids = od.get_device_ids()
id = device_ids[0]

# open device
device = od.open_device(id)

# Get info to check it worked
serialNumber = device.get_serial_number()
print(device.get_model())
# print(device.get_device_type()) 

# to ensure spectrum is always has the maximum number of wavelegnths
num_pixels = device.get_formatted_spectrum_length()

####
# Set parameters
####
num_ave_scans = 5
integrationTime = 50000
wavelength_lo = 450
wavelength_hi = 455
acquisition_delay = 0

####
# Update parameters values
####
# Update integration time to be between physical limits
integrationTime = min(device.get_maximum_integration_time(), 
                    max(device.get_minimum_integration_time(), 
                    device.get_minimum_averaging_integration_time(), 
                    integrationTime,
                        )
                    )

# Update acquisition time to be between physical limits
acquisition_delay = min(device.get_acquisition_delay_maximum(),
                        max(acquisition_delay, 
                            device.get_acquisition_delay_minimum()
                            )
                        )

####
# Set parameters on device
####
# Set number of scans to average
try:
    device.set_scans_to_average(num_ave_scans)
except OceanDirectError as err:
    [errorCode, errorMsg] = err.get_error_details()
    print("set_scans_to_average: exception / %d = %s" % (errorCode, errorMsg))
    
# Set acquisition delay
try:
    device.set_acquisition_delay(acquisition_delay)
except OceanDirectError as err:
    [errorCode, errorMsg] = err.get_error_details()
    print("set_acquisition_delay: exception / %d = %s" % (errorCode, errorMsg))
    
# Set integration time
try:
    device.set_integration_time(integrationTime)
except OceanDirectError as err:
    [errorCode, errorMsg] = err.get_error_details()
    print("set_integration_time: exception / %d = %s" % (errorCode, errorMsg))


####
# Get actual parameters from device
####



####
# Perform measurements
####
# Get spectrum

try:
    spectrum = device.get_formatted_spectrum()

    # List of the indices and wavelengths to be measured
    wls_idx, wls  = device.get_indices_at_wavelength_range(
          wavelength_lo, wavelength_hi, num_pixels
          )

    # List of the intensities at the wavelengths specified
    intensity_vals = [spectrum[i] for i in wls_idx]
except OceanDirectError as e:
        [errorCode, errorMsg] = err.get_error_details()
        print("get_spec_formatted(device): exception / %d = %s" % (errorCode, errorMsg))


#### Need to add store of "dark" spectrum but probably just use SW


# device.set_stored_dark_spectrum(spectra)
# print(device.get_electric_dark_pixel_indices())

od.close_device(id)

OCEANHR6


In [15]:
wls

[450.27999304656095,
 450.74084177821004,
 451.201665838806,
 451.66246522674146,
 452.1232399404092,
 452.5839899782018,
 453.0447153385121,
 453.5054160197327,
 453.96609202025627,
 454.4267433384756,
 454.88736997278335]

In [91]:
# Electric dark pixel correction?
# get_number_electric_dark_pixels
# get_electric_dark_pixel_indices() 



# Dark spectrum - not sure if we want to use spectrometer or do this in software

#  def set_stored_dark_spectrum(self, darkSpectrum: list[float]) -> None:
#         """!
#         Store a dark spectrum for use in subsequent corrections i.e. dark correction and nonlinearity correction.
#         @see getStoredDarkSpectrum.

        
#  def get_stored_dark_spectrum(self) -> list[float]:
#         """!
#         Retrieve a previously stored dark spectrum for use in subsequent corrections i.e. dark correction and nonlinearity correction.



# get_dark_corrected_spectrum1(self, darkSpectrum: list[float]) -> list[float]:
#         """!
#         Acquire a spectrum and use the supplied dark spectrum to perform a dark correction then return the dark corrected spectrum.

# def get_dark_corrected_spectrum2(self) -> list[float]:
#         """!
#         Acquire a spectrum and use the previously stored dark spectrum to perform a dark correction then return the dark corrected spectrum.


27.59814085913352


In [ ]:
# set_integration_time

# get_index_at_wavelength
# get_indices_at_wavelengths
# get_indices_at_wavelength_range
# get_wavelengths

# set_acquisition_delay

# set_stored_dark_spectrum

# set_scans_to_average
# get_scans_to_average

# set_boxcar_width
# get_boxcar_width

# get_max_intensity

# get_formatted_spectrum_length

# get_minimum_integration_time
# get_maximum_integration_time
# get_minimum_averaging_integration_time

# device.Advanced.get_raw_spectrum_with_metadata(spectra, timestamp, 3)

# Advanced.reset_device()